In [5]:
import numpy as np
import math
import random
import timeit
from scipy.special import expit

data = np.genfromtxt('diabetes.csv',delimiter=',')
for k in range(len(data)):
    if data[k][8]==0:
        data[k][8]=-1
        
data_x = data[1:500]
data_t = data[500:]

dtype_ = np.dtype(dtype="float64")

input_data=[]

for r in range(len(data_x)):
    input_data.append([data_x[r][:7],data_x[r][8:]])

testing_data=[]

for s in range(len(data_t)):
    testing_data.append([data_t[s][:7],data_t[s][8:]])

#Setting up the data type for numpy arrays
dtype="float64"
dtype_ = np.dtype(dtype)
x_i = list()
y_i = list()
for i, j in input_data:
        x_i.append(i)
        y_i.append(j)
x_i, y_i = np.array(x_i, dtype=dtype_), np.array(y_i, dtype=dtype_)

#Initializing the values of the Omega and Beta
start_data_w= np.random.randint(np.size(x_i[0]), size=np.size(x_i[0]))
start_data_b = np.random.randint(np.size(y_i))

#Gradient function
values = []
def gradient(training,W,B):
    L = 0.0001
    m = np.size(W)
    a = np.zeros(1)
    s = np.zeros(m+1)
    b = np.append(W,a)

    G0 = 2*L*b
    
    for x,y in input_data:
        c = np.array(np.sum(-y*x), dtype=dtype)
        d = np.array([[-y]], dtype=dtype)
        e = np.array([[c],d], dtype=dtype)

        if np.add(1,np.sum(-y*np.add(np.sum(W*x),B)))<=0:
            values.append([[np.add(G0,s)]])

        elif np.add(1,np.sum(-y*np.add(np.sum(W*x),B)))>0:
            values.append([[np.add(G0,e)]])

def ssgm(gradient, training_d, start_w, start_b, stepsize, tolerance, n_iter, batch_size,
        dtype="float64", random_state=None):
    
    start = timeit.default_timer()
    
    #Checking if the gradient is callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable")

    #Setting up the data type for numpy arrays
    dtype_ = np.dtype(dtype)

    #Converting x and y to numpy arrays
    n_obs = x_i.shape[0]
    if n_obs != y_i.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match")
    x_iy_i = np.c_[x_i.reshape(n_obs, -1), y_i.reshape(n_obs, 1)]
    
    #Initializing the random number generator
    seed = None if random_state is None else int(random_state)
    rng = np.random.default_rng(seed=seed)

    #Initializing the values of the variables
    StartingV = [start_w, start_b]
    vector = np.array(start_w, dtype=dtype_)
    beta = np.array(start_b, dtype=dtype_)

    #Setting up and checking the stepsize
    stepsize = np.array(stepsize, dtype=dtype_)
    
    if np.any(stepsize <= 0):
        raise ValueError("'stepsize' must be greater than zero")

    #Setting up and checking the size of minibatches
    batch_size = int(batch_size)
    if not 0 < batch_size <= n_obs:
        raise ValueError(
            "'batch_size' must be greater than zero and less than "
            "or equal to the number of observations"
        )

    #Setting up and checking the maximal number of iterations
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than zero")

    #Setting up and checking the tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <= 0):
        raise ValueError("'tolerance' must be greater than zero")
    
    #Performing the gradient descent loop
    for _ in range(n_iter):
        #Shuffle x and y
        rng.shuffle(x_iy_i)

        #Performing minibatch moves
        for StartingV in range(0, n_obs, batch_size):
            stop_t = StartingV + batch_size
            x_batch, y_batch = x_iy_i[StartingV:stop_t, :-1], x_iy_i[StartingV:stop_t, -1:]

            #Recalculating the difference
            diff = np.sum(-stepsize,gradient([x_batch, y_batch], vector, beta))

            #Checking if the absolute difference is small enough
            if (abs(diff.all()) > tolerance):
                #Updating the values of the variables
                vector+=diff
                beta+=diff
            else:
                print("break",start,stop)
                break
                
        print(vector,beta)           
    count_accuracy = 0
    
    for h,l in testing_data:
        if np.add(np.sum(np.array(vector)*np.array(h)),np.array(beta))>=0 and np.array(l)==1:
            count_accuracy += 1

        elif np.add(np.sum(np.array(vector)*np.array(h)),np.array(beta))<0 and np.array(l)==-1:
            count_accuracy += 1
            
        check_accuracy = np.divide(count_accuracy,len(testing_data))*100
    
    stop = timeit.default_timer()
    
    return(vector if vector.shape else vector.item(), 
           print("The local minimum occurs at: Omega={}".format(vector), "Beta={}".format(beta)),
          print("Reported accuracy:{}%".format(check_accuracy)), print('Run time is: ', stop-start))

print(ssgm(gradient, input_data, start_data_w, start_data_b, 0.0001, 1e-04, 100, 1))

[4.9501 1.9501 1.9501 5.9501 5.9501 1.9501 3.9501] 42.95009999999834
[4.9002 1.9002 1.9002 5.9002 5.9002 1.9002 3.9002] 42.90019999999669
[4.8503 1.8503 1.8503 5.8503 5.8503 1.8503 3.8503] 42.85029999999503
[4.8004 1.8004 1.8004 5.8004 5.8004 1.8004 3.8004] 42.800399999993374
[4.7505 1.7505 1.7505 5.7505 5.7505 1.7505 3.7505] 42.75049999999172
[4.7006 1.7006 1.7006 5.7006 5.7006 1.7006 3.7006] 42.70059999999006
[4.6507 1.6507 1.6507 5.6507 5.6507 1.6507 3.6507] 42.650699999988404
[4.6008 1.6008 1.6008 5.6008 5.6008 1.6008 3.6008] 42.60079999998675
[4.5509 1.5509 1.5509 5.5509 5.5509 1.5509 3.5509] 42.55089999998509
[4.501 1.501 1.501 5.501 5.501 1.501 3.501] 42.500999999983435
[4.4511 1.4511 1.4511 5.4511 5.4511 1.4511 3.4511] 42.45109999998178
[4.4012 1.4012 1.4012 5.4012 5.4012 1.4012 3.4012] 42.40119999998012
[4.3513 1.3513 1.3513 5.3513 5.3513 1.3513 3.3513] 42.351299999978465
[4.3014 1.3014 1.3014 5.3014 5.3014 1.3014 3.3014] 42.30139999997681
[4.2515 1.2515 1.2515 5.2515 5.2515 1